In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import argparse
import time
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
parser = argparse.ArgumentParser(description='Add these argument for training')
parser.add_argument('--dir', default='results', help='directory for saving trianed mode')
parser.add_argument('--feature', required= True)

parser.add_argument('--lr', default=0.005, help='learning rate')
parser.add_argument('--epochs', default=10, help='epoch number')
parser.add_argument('--batch_size', default=32)
args = parser.parse_args()

## Set parameter for the model
directory = args.dir                # directory for saving model
batch_size = int(args.batch_size)   # batch size
learning_rate = float(args.lr)
num_epochs = int(args.epochs)

### **Data Processing**

In [4]:
data = pd.read_csv('data/solar_wind_parameters_data_1_hourly_all.csv')

data = data.drop(columns=['Unnamed: 0','Timestamp'])

data.head()

,YEAR,DOY,HR,Scalar_B,BX_GSE_GSM,BY_GSE,BZ_GSE,BY_GSM,BZ_GSM,Proton_Density,SW_Plasma_Temperature,SW_Plasma_Speed,Dst-index
0,1999,1,0,6.792,-1.686,2.710,5.118,1.284,5.650,6.070,81042.0,415.28,-7.0
1,1999,1,1,6.884,-4.513,0.390,3.240,-0.361,3.255,7.027,90525.0,400.50,-4.0
2,1999,1,2,7.073,-3.813,-0.826,4.949,-1.808,4.681,6.793,88927.0,405.24,-4.0
3,1999,1,3,6.644,-3.535,1.860,0.244,1.753,0.515,7.129,104190.0,413.07,-8.0
4,1999,1,4,6.645,-3.082,1.953,2.664,1.730,2.866,6.963,101510.0,415.57,-8.0


In [5]:
features = ['Scalar_B', 'BX_GSE_GSM', 'BY_GSE', 'BZ_GSE', 'BY_GSM', 'BZ_GSM', 'Proton_Density', 'SW_Plasma_Temperature', 'SW_Plasma_Speed']
target = ['Dst-index']

# Select the features and target
data = data[features + target]

# Normalize the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Split the data into sequences
def sequence(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) # all columns except the last
        y.append(data[i+seq_length, -1])  # target is the last column
        # print(f'X:{np.array(X)}, Y:{np.array(y)}')
    return np.array(X), np.array(y)

SEQ_LENGTH = 24
X, y = sequence(data_scaled, SEQ_LENGTH)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_load = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_load = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Train shape: torch.Size([169651, 24, 9]), torch.Size([169651])
Test shape: torch.Size([42413, 24, 9]), torch.Size([42413])


In [7]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, input_seq):
        h0 = torch.zeros(1, input_seq.size(1), self.hidden_layer_size).to(self.device)
        c0 = torch.zeros(1, input_seq.size(1), self.hidden_layer_size).to(self.device)

        lstm_out, _ = self.lstm(input_seq, (h0, c0))

        predictions = self.linear(lstm_out)
        return predictions

In [11]:
if __name__ == "__main__":
    input_size = len(features)
    hidden_layer_size = 64
    output_size = 1
    model = LSTM(input_size, hidden_layer_size, output_size).to(device)
    print(f'Model: {model} input_size: {input_size}, hidden_layer_size: {hidden_layer_size}, output_size: {output_size}')
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    train_loss = 0.0
    best_loss = 1e10
    best_epoch = 0
    for epoch in range(num_epochs):
        start = time.time()
        model.train()
        for i, data in enumerate(train_load):
            X_train, y_train = data
            optimizer.zero_grad()
            y_pred = model(X_train)

            loss = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        end = time.time()
            
        train_loss = train_loss / len(train_load)

        # Test the model
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_load):
                X_test, y_test = data
                y_pred = model(X_test)
                loss = criterion(y_pred, y_test)
                test_loss += loss.item()
        test_loss = test_loss / len(test_load)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Time: {end-start:.2f} sec")
    
        if test_loss < best_loss:
            print(f"Model improved from {best_loss:.4f} to {test_loss:.4f} Saving model...")
            best_loss = test_loss
            best_epoch = epoch + 1

            # Save checkpoint
            checkpoint = {
                'epoch': best_epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss
            }
            torch.save(checkpoint, f'{directory}/lstm_{best_epoch}.pt')

    # Save the model
    # save_path = f'{directory}/lstm_{args.feature}.pt'
    save_path = f'{directory}/lstm_1.pt'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    torch.save(model.state_dict(), save_path)
    # torch.save(model.state_dict(), f'{directory}/lstm_{args.feature}.pt')
    print(f"Model saved to {directory}/lstm_1.pt")


Model: LSTM(
  (lstm): LSTM(9, 64)
  (linear): Linear(in_features=64, out_features=1, bias=True)
) input_size: 9, hidden_layer_size: 64, output_size: 1


/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([19])) that is different to the input size (torch.Size([19, 24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 24, 1])). This will likel

Epoch [1/10], Train Loss: 0.0089, Test Loss: 0.0046, Time: 3.30 sec
Model improved from 10000000000.0000 to 0.0046 Saving model...
Epoch [2/10], Train Loss: 0.0084, Test Loss: 0.0045, Time: 3.30 sec
Model improved from 0.0046 to 0.0045 Saving model...
Epoch [3/10], Train Loss: 0.0082, Test Loss: 0.0044, Time: 3.30 sec
Model improved from 0.0045 to 0.0044 Saving model...
Epoch [4/10], Train Loss: 0.0079, Test Loss: 0.0043, Time: 3.28 sec
Model improved from 0.0044 to 0.0043 Saving model...
Epoch [5/10], Train Loss: 0.0077, Test Loss: 0.0043, Time: 3.28 sec
Epoch [6/10], Train Loss: 0.0076, Test Loss: 0.0043, Time: 3.29 sec
Model improved from 0.0043 to 0.0043 Saving model...
Epoch [7/10], Train Loss: 0.0075, Test Loss: 0.0043, Time: 3.28 sec
Model improved from 0.0043 to 0.0043 Saving model...
Epoch [8/10], Train Loss: 0.0074, Test Loss: 0.0043, Time: 3.26 sec
Model improved from 0.0043 to 0.0043 Saving model...
Epoch [9/10], Train Loss: 0.0074, Test Loss: 0.0042, Time: 3.27 sec
Model i